<a href="https://colab.research.google.com/github/Lucasdz-7/Projeto-Integrador-2/blob/main/C%C3%B3digoAPi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os
from google.colab import files
from google.colab import drive

In [3]:
drive.mount('/content/drive')
origem = '/content/drive/MyDrive/Data/Dados/'
DadosIbama = origem + 'dados_ibama.csv'

Mounted at /content/drive


In [4]:
# Criando Dataframes

Dados_IBAMA = pd.read_csv(DadosIbama, low_memory=False, sep=';', encoding='utf-8')

In [ ]:
Dados_IBAMA.head(10)

In [5]:
# Filtrando para o período entre 2013 - 2023 e ordenando por ordem crescente

Dados_IBAMA_2013a2023 = Dados_IBAMA[(Dados_IBAMA["Ano"] >= 2013) & (Dados_IBAMA["Ano"] <= 2023)]
Dados_IBAMA_2013a2023 = Dados_IBAMA_2013a2023.sort_values(by='Ano', ascending=True)

In [ ]:
Dados_IBAMA_2013a2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2941968 entries, 200354 to 281960
Data columns (total 16 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   CNPJ                     object
 1   Razão Social             object
 2   Estado                   object
 3   Município                object
 4   Ano                      int64 
 5   Produto                  object
 6   Quantidade Transportada  object
 7   Unidade de Medida        object
 8   Tipo de Transporte       object
 9   Tipo de Armazenamento    object
 10  Plano de Emergência      object
 11  UF - origem              object
 12  Município - origem       object
 13  UF - destino             object
 14  Município - destino      object
 15  Situação Cadastral       object
dtypes: int64(1), object(15)
memory usage: 381.6+ MB


In [ ]:
Dados_IBAMA_2013a2023.head(5)

In [6]:
# Removendo todos outros modais, preservando apenas o rodoviário.

Dados_IBAMA_teste = Dados_IBAMA_2013a2023.query("`Tipo de Transporte` == 'Rodoviário'")

In [7]:
# Removendo zeros e valores nulos.

Dados_IBAMA_teste = Dados_IBAMA_teste[(Dados_IBAMA_teste != 0).all(axis=1)].dropna()
(Dados_IBAMA_teste == 0).sum()

,0
CNPJ,0
Razão Social,0
Estado,0
Município,0
Ano,0
Produto,0
Quantidade Transportada,0
Unidade de Medida,0
Tipo de Transporte,0
Tipo de Armazenamento,0


In [8]:
# Apenas unidades de medidas de kg e litros

Dados_IBAMA_teste = Dados_IBAMA_teste.query(
    "`Unidade de Medida` in ['kilogramas', 'Litro', 'Metro Cúbico',  'Tonelada']"
)

In [25]:
print(Dados_IBAMA_teste['Unidade de Medida'].unique())

['Tonelada' 'kilogramas' 'Litro' 'Metro Cúbico']


In [ ]:
# 🔸 Lista de palavras-chave para filtrar produtos perigosos ou especiais
# Inclui:
# - Combustíveis e derivados (ex: gasolina, diesel, etanol, GLP, querosene, óleo, lubrificantes)
# - Substâncias químicas perigosas (ex: ácidos, nitratos, solventes, cloro, amônia, formaldeído, etc.)
# - Termos relacionados a risco (inflamável, tóxico, corrosivo, oxidante, radioativo, explosivo...)
# - Resíduos e efluentes industriais (lama, lodo, borra, emulsão, catalisadores, defensivos, fertilizantes químicos)
# - Gases e criogênicos (oxigênio, nitrogênio, CO2, gás comprimido, criogênico...)
# - Metais pesados e compostos correlatos (mercúrio, chumbo, dicromato, etc.)
# - Produtos industriais diversos (tintas, vernizes, betume, adesivos químicos, aditivos, borracha sintética...)

palavras_perigosas = [
    'gasolina', 'diesel', 'etanol', 'álcool', 'biodiesel', 'combustível',
    'querosene', 'gnv', 'gás liquefeito', 'gás natural', 'butano', 'propano',
    'glp', 'naphta', 'nafta', 'óleo', 'lubrificante',

    'ácido', 'base', 'alcalino', 'hidróxido', 'nitrato', 'peróxido',
    'hipoclorito', 'cloro', 'amônia', 'soda cáustica', 'cloreto', 'sulfato',
    'fosfato', 'ácetico', 'acético', 'cianeto', 'anidro', 'solvente',
    'tolueno', 'xileno', 'benzeno', 'acetona', 'formaldeído', 'fenol',
    'butil', 'metanol', 'isopropanol', 'hexano', 'heptano', 'éter',

    'inflamável', 'tóxico', 'corrosivo', 'explosivo', 'oxidante',
    'radioativo', 'perigoso', 'reativo', 'irritante', 'comburente',
    'poluente', 'resíduo perigoso', 'resíduo químico',

    'resíduo', 'lama', 'lodo', 'efluente', 'emulsão', 'borra', 'bagaço químico',
    'catalisador', 'pesticida', 'herbicida', 'inseticida', 'defensivo',
    'fertilizante químico', 'corante industrial', 'pigmento químico',

    'oxigênio', 'nitrogênio', 'hélio', 'argônio', 'co2', 'co', 'freon',
    'refrigerante', 'criogenico', 'criogênico', 'gás comprimido', 'gás liquefeito',

    'mercúrio', 'chumbo', 'cádmio', 'arsênio', 'cobre', 'níquel',
    'zinco', 'manganês', 'crômico', 'cromato', 'dicromato',

    'betume', 'asfalto', 'tinta', 'verniz', 'cola industrial',
    'adesivo químico', 'detergente industrial', 'espessante químico',
    'aditivo químico', 'plástico fundido', 'borracha sintética'
]

mask = Dados_IBAMA_teste['Produto'].str.lower().str.contains('|'.join(palavras_perigosas), na=False)
Dados_IBAMA_teste = Dados_IBAMA_teste[mask]


In [ ]:
contagem_valores = Dados_IBAMA_teste['Produto'].value_counts()
print(contagem_valores)

In [ ]:
Dados_IBAMA_teste['Produto'].value_counts().tail(50)

In [13]:
# Inicializa colunas novas
Dados_IBAMA_teste['qntd_Litros'] = 0.0
Dados_IBAMA_teste['qntd_Kg'] = 0.0

# --- CORREÇÃO: converte strings problemáticas para float antes de normalizar ---
Dados_IBAMA_teste['Quantidade Transportada'] = Dados_IBAMA_teste['Quantidade Transportada'].apply(
    lambda x: float(str(x)) if x not in [None, ''] else 0.0
)


def normalizar(row):
    valor = row['Quantidade Transportada']
    unidade = str(row['Unidade de Medida']).strip()  # mantém maiúsculas/minúsculas conforme estão

    # --- CONVERSÃO PARA LITRO ---
    if unidade in ['Metro Cúbico']:
        row['Quantidade Transportada'] = valor * 1000
        row['Unidade de Medida'] = 'Litro'

    # --- CONVERSÃO PARA KILOGRAMA ---
    elif unidade in ['Tonelada']:
        row['Unidade de Medida'] = 'kilogramas'

    return row

# Aplica a normalização
Dados_IBAMA_teste = Dados_IBAMA_teste.apply(normalizar, axis=1)

# Preenche as colunas finais
Dados_IBAMA_teste.loc[Dados_IBAMA_teste['Unidade de Medida'] == 'Litro', 'qntd_Litros'] = Dados_IBAMA_teste['Quantidade Transportada']
Dados_IBAMA_teste.loc[Dados_IBAMA_teste['Unidade de Medida'] == 'kilogramas', 'qntd_Kg'] = Dados_IBAMA_teste['Quantidade Transportada']

# Remove as colunas antigas
Dados_IBAMA_teste = Dados_IBAMA_teste.drop(columns=['Quantidade Transportada', 'Unidade de Medida'])

# Visualiza resultado
print(Dados_IBAMA_teste[['qntd_Litros','qntd_Kg']].head(10))

        qntd_Litros     qntd_Kg
200354          0.0  30951000.0
200353          0.0  25978000.0
200352          0.0  13190000.0
200288          0.0  34830000.0
200269          0.0  14139000.0
200225          0.0  13840000.0
200121          0.0  31222000.0
198898          0.0  25949000.0
198535          0.0  13960000.0
198498          0.0   7170000.0


In [14]:
Dados_IBAMA_teste.tail(10)

,CNPJ,Razão Social,Estado,Município,Ano,Produto,Tipo de Transporte,Tipo de Armazenamento,Plano de Emergência,UF - origem,Município - origem,UF - destino,Município - destino,Situação Cadastral,qntd_Litros,qntd_Kg
2743289,46.720.027/0001-50,3R COMERCIO E LOGISTICA LTDA,SAO PAULO,RIBEIRAO PRETO,2023,Serviço de produção de álcool ou serviços rela...,Rodoviário,Tanques,Sim,SAO PAULO,GUARULHOS,SAO PAULO,BATATAIS,Ativa,216000.0,0.0
3788396,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Ar líqüido ou ar comprimido,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,IVAIPORA,Ativa,2000.0,0.0
3787825,00.712.503/0001-90,OXBRAS GASES MEDICINAIS E INDUSTRIAIS DO NORDE...,BAHIA,ALAGOINHAS,2023,Oxigênio,Rodoviário,Tanques,Sim,BAHIA,ALAGOINHAS,BAHIA,ARACAS,Ativa,13500000.0,0.0
3787826,00.712.503/0001-90,OXBRAS GASES MEDICINAIS E INDUSTRIAIS DO NORDE...,BAHIA,ALAGOINHAS,2023,Argônio (gases raros),Rodoviário,Tanques,Sim,BAHIA,ALAGOINHAS,BAHIA,RIO REAL,Ativa,37000.0,0.0
3787904,01.112.425/0001-55,MAGMIN LTDA,MINAS GERAIS,ITAUNA,2023,"Explosivos preparados, exceto pólvoras propuls...",Rodoviário,Outros,Sim,MINAS GERAIS,ITAUNA,MINAS GERAIS,DESTERRO DE ENTRE RIOS,Ativa,0.0,2575.0
3788372,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Oxigênio,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,MANDAGUARI,Ativa,415000.0,0.0
3787973,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,"Produtos inorgânicos, n.e, exceto os compostos...",Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,MARINGA,Ativa,603000.0,0.0
3788169,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Acetileno,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,PATO BRANCO,Ativa,0.0,33.0
3788355,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Oxigênio,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,TUNEIRAS DO OESTE,Ativa,190000.0,0.0
281960,09.446.066/0001-84,OXICAM GASES LTDA ME,SAO PAULO,PAULINIA,2023,Oxigênio,Rodoviário,Tanques,Não,ESPIRITO SANTO,VITORIA,SAO PAULO,CAMPINAS,Ativa,0.0,30000.0
